In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

In [4]:
# Generar datos de ventas
np.random.seed(42)

ventas = pd.DataFrame({
    'venta_id': range(1, 1001),
    'cliente_id': np.random.randint(1, 101, 1000),
    'producto_id': np.random.randint(1, 51, 1000),
    'cantidad': np.random.randint(1, 11, 1000),
    'precio_unitario': np.round(np.random.uniform(10, 500, 1000), 2),
    'fecha_venta': pd.date_range('2024-01-01', periods=1000, freq='1h'),
    'updated_at': datetime.now()
})

In [5]:
ventas['total'] = ventas['cantidad'] * ventas['precio_unitario']

In [6]:
print(f"Generados {len(ventas)} registros de ventas")
ventas.head()

Generados 1000 registros de ventas


,venta_id,cliente_id,producto_id,cantidad,precio_unitario,fecha_venta,updated_at,total
0,1,52,34,4,405.14,2024-01-01 00:00:00,2026-01-08 19:44:40.093429,1620.56
1,2,93,47,10,235.03,2024-01-01 01:00:00,2026-01-08 19:44:40.093429,2350.30
2,3,15,8,6,35.46,2024-01-01 02:00:00,2026-01-08 19:44:40.093429,212.76
3,4,72,40,7,395.28,2024-01-01 03:00:00,2026-01-08 19:44:40.093429,2766.96
4,5,61,49,2,108.67,2024-01-01 04:00:00,2026-01-08 19:44:40.093429,217.34


In [7]:
import sqlite3

In [8]:
def carga_completa_sqlite(df, tabla):
    # Conexión a base de datos en memoria
    conn = sqlite3.connect(':memory:')
    
    # Crear tabla
    conn.execute(f'''
        CREATE TABLE {tabla} (
            venta_id INTEGER PRIMARY KEY,
            cliente_id INTEGER,
            producto_id INTEGER,
            cantidad INTEGER,
            precio_unitario REAL,
            total REAL,
            fecha_venta TEXT,
            updated_at TEXT
        )
    ''')
    
    # Insertar datos (reemplazo total)
    df.to_sql(tabla, conn, if_exists='replace', index=False)
    
    # Verificación de carga
    cursor = conn.execute(f"SELECT COUNT(*) FROM {tabla}")
    count = cursor.fetchone()[0]
    
    conn.close()
    return count

In [9]:
registros_cargados = carga_completa_sqlite(ventas, 'ventas_completas')
print(f"Carga completa: {registros_cargados} registros")

Carga completa: 1000 registros


In [10]:
ultimo_id = 500

In [11]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [12]:
def carga_incremental(df, archivo_parquet, ultimo_id=0):
    # Filtrar solo registros nuevos
    nuevos_registros = df[df['venta_id'] > ultimo_id]
    
    if len(nuevos_registros) > 0:
        try:
            nuevos_registros.to_parquet(
                archivo_parquet,
                engine='pyarrow',
                index=False
            )
            print(f"Carga incremental: {len(nuevos_registros)} nuevos registros")
            return len(nuevos_registros)
        except Exception as e:
            print(f"Error en carga incremental: {e}")
            return 0
    else:
        print("No hay nuevos registros para cargar")
        return 0

In [13]:
nuevos_cargados = carga_incremental(
    ventas,
    'ventas_incremental.parquet',
    ultimo_id=500
)

print(f"Registros nuevos agregados: {nuevos_cargados}")

Carga incremental: 500 nuevos registros
Registros nuevos agregados: 500


In [14]:
import time

def comparar_estrategias_carga():
    estrategias = {}
    
    # Medir carga completa
    start = time.time()
    carga_completa_sqlite(ventas, 'ventas_test')
    estrategias['completa'] = time.time() - start
    
    # Medir carga incremental (simulada, solo filtro)
    start = time.time()
    _ = ventas[ventas['venta_id'] > 800]
    estrategias['incremental'] = time.time() - start
    
    print("Comparación de estrategias:")
    print(f"Carga completa: {estrategias['completa']:.4f} segundos")
    print(f"Carga incremental: {estrategias['incremental']:.4f} segundos")
    
    return estrategias

resultados = comparar_estrategias_carga()

Comparación de estrategias:
Carga completa: 0.0129 segundos
Carga incremental: 0.0014 segundos


In [15]:
import pandas as pd
import time

# ---------------------------
# Parámetros
# ---------------------------
ultimo_id = 800
archivo_excel = "evidencia_carga_dia4.xlsx"

# ---------------------------
# Dataset incremental
# ---------------------------
ventas_incrementales = ventas[ventas['venta_id'] > ultimo_id].copy()

# ---------------------------
# Medición de tiempos
# ---------------------------
start = time.time()
_ = carga_completa_sqlite(ventas, 'ventas_test')
tiempo_completa = time.time() - start

start = time.time()
_ = ventas_incrementales.copy()  # Simulación de carga incremental
tiempo_incremental = time.time() - start

# ---------------------------
# Resumen de estrategias
# ---------------------------
resumen_carga = pd.DataFrame({
    'estrategia': ['carga_completa', 'carga_incremental'],
    'registros_procesados': [len(ventas), len(ventas_incrementales)],
    'tiempo_segundos': [round(tiempo_completa, 4), round(tiempo_incremental, 4)]
})

# ---------------------------
# Escritura a Excel
# ---------------------------
with pd.ExcelWriter(archivo_excel, engine='openpyxl') as writer:
    ventas.to_excel(writer, sheet_name='ventas_originales', index=False)
    ventas_incrementales.to_excel(writer, sheet_name='ventas_incrementales', index=False)
    resumen_carga.to_excel(writer, sheet_name='resumen_carga', index=False)

print(f"Archivo de evidencia generado correctamente: {archivo_excel}")

Archivo de evidencia generado correctamente: evidencia_carga_dia4.xlsx
